# Aula 9 - ReAct

Leandro Carísio Fernandes

## Setup

In [177]:
TOTAL_PERGUNTAS = 150

DIR_AULA9 = "/content/drive/My Drive/IA024A-Redes_Neurais_NLP/Aula9-ReAct/"

ARQUIVO_PARA_TESTAR = f'{DIR_AULA9}test_questions.json'

# Se True, gera o índice invertido e salva em um pickle. Se False, apenas recupera o pickle do drive
GERAR_INDICE_INVERTIDO = False
ARQUIVO_INDICE_INVERTIDO = f'{DIR_AULA9}iidx_artigos_segmentados.pickle'

# Se True, gera as respostas. Se False, apenas recupera o pickle do drive
AGREGAR_DOCUMENTOS = False
ARQUIVO_AGREGADO = 'respostas_obtidas.pickle'

In [107]:
from getpass import getpass
GROQ_API = getpass("API groq")

API groq··········


In [108]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
!pip install Groq
!pip install unidecode

# Download da base de dados

Link para a base IIRC: https://github.com/jferguson144/IIRC-baseline

Baixa arquivo iirc.tar.gz e context_articles.tar.gz conforme setup.sh do IIRC: https://github.com/jferguson144/IIRC-baseline/blob/main/setup.sh

Os casos de teste foram selecionados previamente pelo professor. Estão no arquivo do drive.


As bases de treino/dev/teste e contextos estão aqui:

    !wget http://jamesf-incomplete-qa.s3.amazonaws.com/iirc.tar.gz
    !tar -xzf iirc.tar.gz
    !rm iirc.tar.gz
    !cd iirc

    !wget http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
    !tar -xzf context_articles.tar.gz
    !rm context_articles.tar.gz

    !wget https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json


Para esse caderno, vamos precisar apenas da base de contexto e teste. Mas os testes foram selecionados pelo professor, então vamos pegar o arquivo com o subset de treino do drive.


In [110]:
!wget http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
!tar -xzf context_articles.tar.gz
!rm context_articles.tar.gz

!cp "{ARQUIVO_PARA_TESTAR}" "./test_questions.json"

--2024-05-14 11:56:46--  http://jamesf-incomplete-qa.s3.amazonaws.com/context_articles.tar.gz
Resolving jamesf-incomplete-qa.s3.amazonaws.com (jamesf-incomplete-qa.s3.amazonaws.com)... 52.218.237.211, 52.92.154.225, 52.218.221.235, ...
Connecting to jamesf-incomplete-qa.s3.amazonaws.com (jamesf-incomplete-qa.s3.amazonaws.com)|52.218.237.211|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 385263479 (367M) [application/x-gzip]
Saving to: ‘context_articles.tar.gz’

context_articles.ta 100%[===================>] 367.42M  60.0MB/s    in 6.1s    

2024-05-14 11:56:53 (60.3 MB/s) - ‘context_articles.tar.gz’ saved [385263479/385263479]



In [111]:
import json

test_set = json.load(open('test_questions.json', 'r'))
context_articles = json.load(open("context_articles.json",'r'))

## Entendendo o dataset

context_articles são os artigos que deverão ser consultados. É um dict onde a chave é o título do artigo. Por exemplo:

In [112]:
print(f'Total de artigos: {len(context_articles)}')
print('5 títulos quaisquer:')
print(list(context_articles.keys())[0:5])

Total de artigos: $56550
5 títulos quaisquer:
['san diego padres', 'john bruce (historiographer)', 'jack nicholson', 'british raj', 'dabu county']


O conjunto de teste tem um monte de perguntas e respostas. Ele indica a pergunta, a resposta esperada e de onde aquela resposta veio (nota: o formato desse arquivo está ligeiramente diferente do formato original). Por exemplo:

In [113]:
i = 0 # i = 3 # Dá pra testar isso com qualquer tipo de resposta span. Para outros tipos, tem que adaptar
print('Primeira pergunta do conjunto de teste:')
print('Pergunta:', test_set[i]['question'])
print('Resposta:', test_set[i]['answer']['answer_spans'][0]['text']) # Isso só funciona pois já sabemos que é do tipo span
print('De onde veio a resposta:', test_set[i]['answer']['answer_spans'][0]['passage'])
print('---------------------')
print('Artigo no context_articles:')
context_articles[test_set[i]['answer']['answer_spans'][0]['passage']]

Primeira pergunta do conjunto de teste:
Pergunta: What is Zeus know for in Greek mythology?
Resposta: sky and thunder god
De onde veio a resposta: zeus
---------------------
Artigo no context_articles:


'Zeus (British English , North American English ; , <i>Zeús</i> ) is the <a href="sky%20father">sky</a> and <a href="thunder%20god">thunder god</a> in <a href="ancient%20Greek%20religion">ancient Greek religion</a>, who rules as <a href="king%20of%20the%20gods">king of the gods</a> of <a href="Mount%20Olympus">Mount Olympus</a>. His name is <a href="cognate">cognate</a> with the first element of his <a href="ancient%20Roman%20religion">Roman</a> <a href="interpretatio%20graeca">equivalent</a> <a href="Jupiter%20%28mythology%29">Jupiter</a>. His mythologies and powers are similar, though not identical, to those of Indo-European deities such as <a href="Jupiter%20%28mythology%29">Jupiter</a>, <a href="Perk%C5%ABnas">Perkūnas</a>, <a href="Perun">Perun</a>, <a href="Indra">Indra</a> and <a href="Thor">Thor</a>.\n\nZeus is the child of <a href="Cronus">Cronus</a> and <a href="Rhea%20%28mythology%29">Rhea</a>, the youngest of his siblings to be born, though sometimes reckoned the eldest as 

## Organizando os dados

Organizando o conjunto de teste para gerar um array de pergunta e resposta esperada.

Imprime as 10 primeiras perguntas e respostas para checar.

Daqui dá pra pegar as 150 questões e respostas pra gente testar no final.

In [114]:
perg_resposta_esperada = []

for i in range(len(test_set)):
  pr = test_set[i]

  pergunta = pr['question']
  resposta_obj = pr['answer']
  tipo_resposta = resposta_obj['type']

  if tipo_resposta == 'binary':
    resposta = resposta_obj['answer_value']
  elif tipo_resposta == 'value':
    resposta = resposta_obj['answer_value'] + ' ' + resposta_obj['answer_unit']
  elif tipo_resposta == 'span':
    resposta = resposta_obj['answer_spans'][0]['text']
  elif tipo_resposta == 'none':
    resposta = 'none'
  else:
    resposta = 'Não pode chegar aqui, os tipos parece que são só binary/value/span/none!'
    print(tipo_resposta)

  perg_resposta_esperada.append({"pergunta": pergunta, "resposta": resposta.strip()})

  if i < 60:
    print(f"{i}: {pergunta}\n{resposta}")
    print('----------------------------------------------------------------------')



0: What is Zeus know for in Greek mythology?
sky and thunder god
----------------------------------------------------------------------
1: How long had the First World War been over when Messe was named aide-de-camp?
5 years
----------------------------------------------------------------------
2: How old was Messe when the First World War started?
30 years
----------------------------------------------------------------------
3: How long had Angela Scoular been acting professionally when she appeared in the movie "On Her Majesty's Secret Service"?
2 years
----------------------------------------------------------------------
4: What is the capacity of the stadium where Brunt returned to action after a torn ACL?
26,688
----------------------------------------------------------------------
5: Which stadium where Brunt played can hold more people, The Hawthorns or White Hart Lane?
White Hart Lane
----------------------------------------------------------------------
6: In which country w

# BM25

## Tratando os dados que serão indexados

Sobre a base IIRC:

"Contains more than 13K questions over paragraphs from English Wikipedia that provide only partial information to answer them, with the missing information occurring in one or more linked documents. The questions were written by crowd workers who did not have access to any of the linked documents, leading to questions that have little lexical overlap with the contexts where the answers appear." [https://paperswithcode.com/dataset/iirc]

Assim, para a base de testes, vamos indexar todos os documentos relacionados à pergunta (ou seja, o documento e os linkados).

In [115]:
test_set[0]

{'answer': {'type': 'span',
  'answer_spans': [{'text': 'sky and thunder god',
    'passage': 'zeus',
    'type': 'answer',
    'start': 83,
    'end': 102}]},
 'question': 'What is Zeus know for in Greek mythology?',
 'context': [{'text': 'he Palici the sons of Zeus',
   'passage': 'main',
   'indices': [684, 710]},
  {'text': 'in Greek mythology', 'passage': 'main', 'indices': [137, 155]},
  {'text': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion',
   'passage': 'Zeus',
   'indices': [0, 110]}],
 'question_links': ['Greek mythology', 'Zeus'],
 'title': 'Palici'}

In [116]:
%%time
documents = []
all_titles = []

for item in test_set:
    for link in item["question_links"]:
        link_title = link.lower()
        if link_title in context_articles and link_title.lower() not in all_titles:
            documents.append({
                "title": link_title,
                "content": context_articles[link_title]
            })
            all_titles.append(link_title)

CPU times: user 360 µs, sys: 0 ns, total: 360 µs
Wall time: 2.38 ms


In [117]:
len(all_titles)

62

Limpa o HTML dos documentos:

In [118]:
import re

# Remover o html igual está na base IIRC é bem mais rápido que usar o BeuatifulSoup
# https://github.com/jferguson144/IIRC-baseline/blob/main/util.py
def remove_html(html):
  return re.sub("<[^>]*>", "", html).strip()

artigos_sem_html = []

# Se for usar só os documentos que estão na base de teste
for doc in documents:
  titulo = doc['title']
  conteudo = doc['content']
  artigos_sem_html.append({"title": titulo, "content": remove_html(conteudo)})

Quebra cada artigo em grupos de 3 sentenças. Usa a biblioteca Spacy para separar as sentenças.

Código de origem: https://github.com/neuralmind-ai/visconde/blob/main/iirc_create_indices.ipynb

In [119]:
%%time
from tqdm import tqdm
import spacy
import pickle

nlp = spacy.blank("en")
nlp.add_pipe("sentencizer")

stride = 2
max_length = 3

def window(documents, stride=2, max_length=3):
  treated_documents = []

  for j, document in enumerate(tqdm(documents)):
    doc_text = document['content']
    doc = nlp(doc_text)
    sentences = [sent.text.strip() for sent in doc.sents]
    for i in range(0, len(sentences), stride):
      segment = ' '.join(sentences[i:i + max_length])
      treated_documents.append({
          "title": document['title'],
          "contents": document['title']+". "+segment,
          "segment": segment
      })
      if i + max_length >= len(sentences):
        break
  return treated_documents


artigos_segmentados = window(artigos_sem_html)

100%|██████████| 62/62 [00:13<00:00,  4.64it/s]

CPU times: user 6.23 s, sys: 18.1 ms, total: 6.25 s
Wall time: 14.6 s


In [120]:
len(artigos_segmentados)

5661

## Implementação do BM25

In [121]:
from collections import Counter
from tqdm import tqdm
import array
import math
import pickle
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from unidecode import unidecode
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')

def tokenizador_en(texto):
    # Remove acentuação e converte para minúsculo
    texto = unidecode(texto.lower())

    # Remove pontuação
    texto = ''.join([char if char not in string.punctuation else ' ' for char in texto])

    # Tokeniza o texto
    tokens = word_tokenize(texto, language='english')

    # Remove stopwords e aplica stemização
    stemmer = PorterStemmer()
    tokens_processados = [stemmer.stem(token) for token in tokens if token not in stopwords.words('english')]

    return tokens_processados

# Definição de uma classe para índice invertido
class IndiceInvertido:

    # Recebe 'tokenizar', uma função tokenizadora. Por padrão, é a função tokenizador_pt
    def __init__(self, tokenizar=tokenizador_en):
        # Cria um índice invertido vazio.
        # A chave é um token e o valor é um objeto contendo a lista de ids de documento que contém o token
        # e uma lista de total de ocorrências do token naquele documento
        self.indice = {}
        # Cria um índice de tamanho de documentos vazio.
        # A chave é a id do documento e o valor é o total de tokens do documento
        self.tamanho_doc = {}
        # Guarda o total de documentos adicionados. É o mesmo que len(self.tamanho_doc.keys())
        self.n_docs = 0
        # Tokenizador. Essa função é aplicada para a extração dos tokens
        # de um documento. Ela deve fazer todo o pré-processamento (conversão para minúsculo,
        # remoção de stop words, lematização etc)
        self.tokenizar = tokenizar

    def adiciona_dataframe(self, df, extrai_id_conteudo_row):
        for index, row in tqdm(df.iterrows(), total=len(df)):
            id_doc, conteudo_doc = extrai_id_conteudo_row(row)
            self.adiciona_doc(id_doc, conteudo_doc)

    def adiciona_objetos(self, lista_objetos, extrai_id_conteudo):
        for objeto in tqdm(lista_objetos):
            id_doc, conteudo_doc = extrai_id_conteudo(objeto)
            self.adiciona_doc(id_doc, conteudo_doc)

    def adiciona_doc(self, id_doc, conteudo_doc=""):
        # Extrai os tokens de um documento
        tokens = self.tokenizar(conteudo_doc)
        # Conta quantas vezes cada token aparece no documento
        contador_tokens_no_documento = Counter(tokens)

        for token, n_ocorrencias in contador_tokens_no_documento.items():
            # Se o token ainda não está no índice, cria
            self.indice.setdefault(token, {"id_doc": [], "n_ocorrencias": array.array("L", [])})
            # Popula as informações do token para esse documento (a id do documento e
            # quantas vezes o token apareceu no documento)
            self.indice[token]['id_doc'].append(id_doc)
            self.indice[token]['n_ocorrencias'].append(n_ocorrencias)

        # Adiciona um documento a n_docs
        self.n_docs += 1
        # Salva o tamanho do documento
        self.tamanho_doc[id_doc] = len(tokens)

    # Funções utilitárias
    def get_tamanho_medio_docs(self):
        # Utilitário para calcular o tamanho médio dos documentos no índice
        return sum(self.tamanho_doc.values()) / self.n_docs

    def total_docs_por_token(self, token):
        # Utilitário para retornar o total de documentos que contém o token
        return len(self.indice[token]['id_doc'])

    def to_pickle(self, nome_arquivo):
        # Para salvar o índice num arquivo, precisamos manter
        # self.indice, self.tamanho_doc e self.n_docs
        obj_para_salvar = {"indice": self.indice, "tamanho_doc": self.tamanho_doc, "n_docs": self.n_docs}
        with open(nome_arquivo, 'wb') as f:
            pickle.dump(obj_para_salvar, f)

    def from_pickle(self, nome_arquivo):
        # Fazemos o caminho inverso aqui:
        with open(nome_arquivo, 'rb') as f:
            obj_recuperado = pickle.load(f)
        self.indice = obj_recuperado['indice']
        self.tamanho_doc = obj_recuperado['tamanho_doc']
        self.n_docs = obj_recuperado['n_docs']

class BM25:
    # A consideração que é feita aqui é que primeiro é criado um índice invertido
    # e populado e, depois, é instanciado um BM25 com esse índice invertido.
    # A ideia é que o indiceInvertido mantém só a relação de tokens e documentos.
    # O BM25 é que faz o cálculo do score. Assim, um indice invertido
    # pode ser compartilhado por mais de um BM25 com configurações distintas
    # (útil para testar diferentes k1 e b, por exemplo, sem ter que reindexar tudo)
    def __init__(self, indice_invertido=IndiceInvertido(), k1 = 0.9, b = 0.4, bias_idf = 0):
        self.bias_idf = bias_idf
        self.k1 = k1
        self.b = b
        # Para setar o índice é necessário já ter a informação
        self.set_indice_invertido(indice_invertido)

    def set_params(self, k1=None, b=None, bias_idf=None):
        # Se mudar qualquer parâmetro, tem que reiniciar o score
        # Se mudar o bias_idf, é necessário recalcular o idf
        if k1 is not None:
            self.k1 = k1
        if b is not None:
            self.b = b
        if bias_idf is not None:
            self.bias_idf = bias_idf
            self.precalcula_idf()

        self.reinicia_score_dos_indices()

    def set_indice_invertido(self, indice_invertido):
        # Quando é adicionado um índice invertido, é necessário reiniciar todo o score/idf calculado
        self.indice_invertido = indice_invertido
        # Guarda o tamanho médio dos documentos do índice
        self.avgdl = self.indice_invertido.get_tamanho_medio_docs()
        # Pré-calcula o idf de cada token
        self.precalcula_idf()
        self.reinicia_score_dos_indices()

    def reinicia_score_dos_indices(self):
        self.score_por_token = {}

    def precalcula_idf(self):
        # Vamos criar um mapa para guardar os idfs pré-calculados para cada token
        self.idf_por_token = {}

        # Número de documento do corpus está presente no objeto indice_invertido
        N = self.indice_invertido.n_docs

        # Varre todos os tokens do índice. Os tokens são as chaves do indice_invertido.indice
        for token in self.indice_invertido.indice.keys():
            # Recupera o total de documentos que tem o token
            n_doc_token = self.indice_invertido.total_docs_por_token(token)

            # Isso já é o suficiente pra calcular o idf (math.log é o logaritmo na base e)
            idf_token = math.log( ((N - n_doc_token + 0.5)/(n_doc_token + 0.5)) + self.bias_idf )
            # E agora, vamos guardar essa informação
            self.idf_por_token[token] = idf_token

    def calcula_score_para_um_token_e_salva(self, token):
        # O cálculo do BM25 para determinada query é a multiplicação do idf pela frequência do termo no documento * (k1 + 1)
        # Além disso, é dividido pela frequencia do termo no documento + k1 * (1 - b + b * tamanho_doc/avgdl)
        idf = self.idf_por_token[token]
        # Juntando tudo, podemos calcular o score pelo BM25
        zip_id_freq = zip(self.indice_invertido.indice[token]['id_doc'], self.indice_invertido.indice[token]['n_ocorrencias'])
        bm25 = array.array("f", [ idf * freq_token_no_doc * (self.k1 + 1) / (freq_token_no_doc + self.k1 * (1 - self.b + self.b * self.indice_invertido.tamanho_doc[id_doc] / self.avgdl)) for (id_doc, freq_token_no_doc) in zip_id_freq ])
        # Salva o bm25 no índice
        self.score_por_token[token] = bm25

    def tokenizar(self, query):
        return self.indice_invertido.tokenizar(query)

    def pesquisar(self, query):
        # Tokeniza a query
        tokens = self.tokenizar(query)

        # Se não tem token para ser pesquisado, retorna conjunto vazio
        if (len(tokens) == 0):
            return []

        # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
        docs_retornado_com_score = Counter({})

        # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
        for token in tokens:
            # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer,
            # entende-se que a frequência desse token em qualquer documento é 0, já que não pode ser encontrado
            if token not in self.indice_invertido.indice:
                continue

            # Pega a lista de documentos que será analisado
            docs_que_tem_token = self.indice_invertido.indice[token]['id_doc']

            # Se for a primeira vez que esse token é pesquisado, é necessário calcular o score relacionado
            # a ele e salvar. Se já tiver sido feito antes, já podemos buscar o cálculo pronto (que funciona
            # como um cache. Isso é útil no caso de várias pesquisas seguidas)
            if token not in self.score_por_token.keys():
                self.calcula_score_para_um_token_e_salva(token)
            score_dos_docs_deste_token = self.score_por_token[token]

            # Agora já temos calculado o score de todos os documentos desse token. Só adiciona ao acumulador de score atual
            # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
            for id_doc, score_par_doc_token in zip(docs_que_tem_token, score_dos_docs_deste_token):
                docs_retornado_com_score[id_doc] += score_par_doc_token

        # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
        docs_com_score = list(docs_retornado_com_score.items())

        # E ordena do mais relevante para o menos relevante
        return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


Testa uma tokenização qualquer:

In [122]:
iidx = IndiceInvertido()
iidx.tokenizar('What is Zeus know for in Greek mythology?')

['zeu', 'know', 'greek', 'mytholog']

Agora efetivamente indexa os artigos segmentados:

In [123]:
%%time
iidx = IndiceInvertido()

if GERAR_INDICE_INVERTIDO:
  # Vamos usar o próprio índice como um id para o segmento. Vai facilitar a vida depois
  # art_seg['contents'] tem o título e 3 frases do artigo
  for id, art_seg in tqdm(enumerate(artigos_segmentados), total=len(artigos_segmentados)):
    iidx.adiciona_doc(id, art_seg['contents'])

  iidx.to_pickle(ARQUIVO_INDICE_INVERTIDO)
else:
  iidx.from_pickle(ARQUIVO_INDICE_INVERTIDO)

CPU times: user 64.9 ms, sys: 12.9 ms, total: 77.8 ms
Wall time: 87.9 ms


Agora sim cria o buscador BM25 e testa:

In [124]:
buscador = BM25(iidx, 0.82, 0.68, 1)

Faz um teste pesquisando a primeira pergunta e exibindo os 5 primeiros resultados do BM25.

In [125]:
print('Pergunta e resposta esperada: ')
print(perg_resposta_esperada[0])

resultado_bm25 = buscador.pesquisar(perg_resposta_esperada[0]['pergunta'])

print('-----------------------------------------------------------------')
print('5 primeiros conteúdos retornados pelo BM25\n')
for i in range(5):
  segmento_id = resultado_bm25[i][0]
  score = resultado_bm25[i][1]

  title = artigos_segmentados[segmento_id]['title']
  contents = artigos_segmentados[segmento_id]['contents']
  segment = artigos_segmentados[segmento_id]['segment']

  print(f'Segmento_id: {segmento_id}. Score: {score}')
  print(f"Title: {title}")
  print(f"Contents: {contents}")
  print('-----------------------------------------------------------------')

Pergunta e resposta esperada: 
{'pergunta': 'What is Zeus know for in Greek mythology?', 'resposta': 'sky and thunder god'}
-----------------------------------------------------------------
5 primeiros conteúdos retornados pelo BM25

Segmento_id: 293. Score: 14.332436084747314
Title: zeus
Contents: zeus. - Rohde, Erwin, Psyche: The Cult of Souls and Belief in Immortality among the Greeks, 1925. - Smith, William, Dictionary of Greek and Roman Biography and Mythology, 1870, Ancientlibrary.com, William Smith, Dictionary: "Zeus" Ancientlibrary.com

External links.- Greek Mythology Link, Zeus stories of Zeus in myth
- Theoi Project, Zeus summary, stories, classical art
- Theoi Project, Cult Of Zeus cult and statues
- Photo: Pagans Honor Zeus at Ancient Athens Temple from National Geographic
-----------------------------------------------------------------
Segmento_id: 139. Score: 14.109086990356445
Title: greek mythology
Contents: greek mythology. In Ancient Roman times, a new Roman mytholo

# Criação de um agente de pesquisa

In [126]:
buscador = BM25(iidx, 0.82, 0.68, 1)

def agente_pesquisa(query, total_hits=5):
  retorno = ''
  resultado_bm25 = buscador.pesquisar(query)

  total_hits = min(total_hits, len(resultado_bm25))

  for i in range(total_hits):
    segmento_id = resultado_bm25[i][0]
    score = resultado_bm25[i][1]

    title = artigos_segmentados[segmento_id]['title']
    segment = artigos_segmentados[segmento_id]['segment']

    retorno += f'Document {i+1}. [{title}] {segment}\n\n'

  if (retorno.strip() == ''):
    retorno = f'Could not find [{query}]'

  return f'[Observation] {retorno}'

In [127]:
perg_resposta_esperada[1]

{'pergunta': 'How long had the First World War been over when Messe was named aide-de-camp?',
 'resposta': '5 years'}

In [128]:
print(agente_pesquisa('Messe aide-de-camp'))

[Observation] Document 1. [monty python's life of brian] Brian's mother protests, telling the crowd that "He's not the Messiah; he's a very naughty boy," and, "There's no Messiah in here. There's a mess, all right, but no Messiah." All of her attempts at dispersing the crowd are rebuffed.

Document 2. [world war ii] The Soviet Gulag became a de facto system of deadly camps during 1942–43, when wartime privation and hunger caused numerous deaths of inmates, including foreign citizens of Poland and other countries occupied in 1939–40 by the Soviet Union, as well as Axis POWs. By the end of the war, most Soviet POWs liberated from Nazi camps and many repatriated civilians were detained in special filtration camps where they were subjected to NKVD evaluation, and 226,127 were sent to the Gulag as real or perceived Nazi collaborators. Japanese prisoner-of-war camps, many of which were used as labour camps, also had high death rates.

Document 3. [world war ii] In addition to concentration c

# Agregação com Llama 3 e ReAct

Definição da mensagem de sistema:

In [129]:
content_system_message = """You are a helpful assistant that answers users' questions. For this, you have access to a BM25 search engine. When you receive a question, you must check the message history and decide if you have enough information to provide an answer or not. If you do not, you can call the search tool by informing the search terms.

All interaction between you and the user must follow standardized rules. Initially, the user will ask a question preceded by the instruction [Question]. Then, you will analyze the message history and decide whether to call the search tool or respond. The reasoning should be preceded by the instruction [Thought]. After the reasoning, you will inform the command preceded by the instruction [Action]. There are only two available commands:

- Search [query]: In this command, you will inform the search term (query) that should be used in the search.
- Finish [answer]: In this command, you will inform the final answer, which should always be short and direct.

The search tool will respond with a list of documents. The response will be preceded by [Observation].

If you need to search more than once, you MUST change the search terms.

Consider the following model of interaction between the user and you. This model should be followed:

USER:
[Question] Aside from the Apple Remote, what other device can control the program Apple Remote was originally designed to interact with?

ASSISTANT:
[Thought] I need to search for Apple Remote and find the program it was originally designed to interact with.
[Action] Search [Apple Remote]

USER:
[Observation] The Apple Remote is a remote control introduced in October 2005 by Apple, originally designed to control the Front Row Media center program.

ASSISTANT:
[Thought] Apple Remote was originally designed to control the Front Row media center program. I need to search for Front Row next and find what other device can control it.
[Action] Search [Front Row]

USER:
[Observation] Could not find [Front Row].

ASSISTANT:
[Thought] Front Row is not found. I need to search for Front Row Software.
[Action] Search [Front Row software]

USER:
[Observation] Front Row is a discontinued media center software.

ASSISTANT:
[Thought] Front Row (software) is controlled by an Apple Remote or the keyboard function keys. So the answer is keyboard function keys.
[Action] Finish [keyboard function keys]
""".strip()

system_message = {
    "role": "system",
    "content": content_system_message
}

Agora chamar o Llama e pegar a resposta:

In [140]:
def extrai_acao(mensagem):
  try:
    action_index = mensagem.rindex("[Action]") + len("[Action]")
    if action_index != -1:
      action_content = mensagem[action_index:].strip()
      action_parts = action_content.split("[", 1)

      if len(action_parts) > 1:
        action = action_parts[0].strip()
        parameters = action_parts[1].rstrip(']')

        return action, parameters
      else:
        return 'Finish', 'none'

  except:
    return 'Finish', 'none'

teste = """
[Thought] I don't have enough information to answer this question. I need to search for Messe and find out when he was named aide-de-camp and when the First World War ended.
[Action] Search [Messelk aslfk laskf laks]
""".strip()

print(extrai_acao(teste))

('Search', 'Messelk aslfk laskf laks')


In [156]:
from groq import Groq
import time

client = Groq(api_key=GROQ_API)

def chat_completion_react(pergunta=None, mensagens_anteriores=[system_message], max_mensagens=10, debug=True, sleep=10):
    mensagens = mensagens_anteriores[:] # Copia o array

    # Se tiver pergunta, é a primeira interação. Adiciona a pergunta.
    if pergunta is not None:
      mensagens.append({
        "role": "user",
        "content": f"[Question] {pergunta}"
      })
      if debug:
        print(f'Pergunta: {pergunta}')
    else:
      # Se não tiver pergunta, devemos verificar a última mensagem e ver qual é a ação
      # a ser feita.
      content_ultima_mensagem = mensagens_anteriores[len(mensagens_anteriores)-1]['content']
      acao, conteudo_acao = extrai_acao(content_ultima_mensagem)
      if debug:
        print(f'Ação: {acao}. Parâmetros: {conteudo_acao}')

      # Se a ação for de busca, precisamos chamar o agente, pegar o retorno
      # e colocar o conteúdo dele aqui na mensagem
      if acao.lower() == 'search':
        mensagens.append({
            "role": "user",
            "content": agente_pesquisa(conteudo_acao)
        })
      # Se a ação for de finalizar, basta retornar o conteúdo da mensagem
      # Para efeitos de debug do exercício, vamos encaminhar também
      # a estrutura de mensagens
      elif acao.lower() == 'finish':
        return conteudo_acao, mensagens
      # Nesse exercício as únicas ações são search e finish. Teoricamente não
      # pode chegar aqui. Se chegar, exibe uma mensagem de erro e retorna
      # com um erro também
      else:
        if debug:
          print('[ERRO]: Ação não prevista')
        return "[ERRO]: Ação não prevista", mensagens

    # Se chegou até aqui e max_mensagens for 0, é porque já chamou ele o
    # número máximo de vezes especificado (default=10). Nesse caso,
    # retorna com uma mensagem de erro informando que não foi possível
    # resolver a questão.
    # Na hora de retornar as mensagens antigas, desconsidera a última que foi inserida,
    # pois ela ainda não foi chamada
    if max_mensagens <= 0:
      if debug:
        print("[ERRO]: Não foi possível responder a pergunta")
      return "[ERRO]: Não foi possível responder a pergunta", mensagens[0:len(mensagens)-1]


    # Se chegou aqui, é porque foi adicionado uma mensagem de usuário
    # (a primeira pergunta ou a resposta do agente)
    # Então chamamos o chat_completion:
    chat_completion = client.chat.completions.create(
        messages=mensagens,
        model="llama3-70b-8192",
        temperature=0,
        max_tokens=32768,
        top_p=1,
        stop=None,
        stream=False,
    )
    texto_resposta = chat_completion.choices[0].message.content
    mensagens.append({
      "role": "assistant",
      "content": texto_resposta
    })

    time.sleep(sleep)
    # Agora chama recursivamente essa função
    return chat_completion_react(pergunta=None, mensagens_anteriores=mensagens, max_mensagens=max_mensagens-1, debug=debug, sleep=sleep)

Faz um teste:

In [157]:
resposta, historico_chat = chat_completion_react(perg_resposta_esperada[0]['pergunta'], max_mensagens=5, sleep=10)

print(resposta)

historico_chat

Pergunta: What is Zeus know for in Greek mythology?
Ação: Search. Parâmetros: Zeus Greek mythology
Ação: Search. Parâmetros: Zeus powers
Ação: Finish. Parâmetros: King of the gods of Mount Olympus and god of the sky and thunder, known for his power to hurl lightning bolts
King of the gods of Mount Olympus and god of the sky and thunder, known for his power to hurl lightning bolts


[{'role': 'system',
  'content': "You are a helpful assistant that answers users' questions. For this, you have access to a BM25 search engine. When you receive a question, you must check the message history and decide if you have enough information to provide an answer or not. If you do not, you can call the search tool by informing the search terms.\n\nAll interaction between you and the user must follow standardized rules. Initially, the user will ask a question preceded by the instruction [Question]. Then, you will analyze the message history and decide whether to call the search tool or respond. The reasoning should be preceded by the instruction [Thought]. After the reasoning, you will inform the command preceded by the instruction [Action]. There are only two available commands:\n\n- Search [query]: In this command, you will inform the search term (query) that should be used in the search.\n- Finish [answer]: In this command, you will inform the final answer, which should always

In [131]:
perg_resp_esperada_obtida = []

In [132]:
from tqdm import tqdm

def salvar_perg_resp_esperada_obtida(perg_resp_esperada_obtida):
  nome_arquivo = f"{DIR_AULA9}{ARQUIVO_AGREGADO}"
  with open(nome_arquivo, 'wb') as f:
    pickle.dump(perg_resp_esperada_obtida, f)

def recuperar_perg_resp_esperada_obtida():
  nome_arquivo_base = f"{DIR_AULA9}{ARQUIVO_AGREGADO}"
  perg_respostas = []
  with open(nome_arquivo_base, 'rb') as f:
    perg_respostas = pickle.load(f)
  return perg_respostas

In [164]:
import time
import json

# Do jeito que foi implementado, se der problema (rate limit), basta executar a
# célula novamente. Ele vai continuar de onde parou
if AGREGAR_DOCUMENTOS:
  for i, perg_resp in enumerate(perg_resposta_esperada):
    print(f'******************* {i}/{len(perg_resposta_esperada)} *******************')
    pergunta = perg_resp['pergunta']
    resposta_esperada = perg_resp['resposta']

    # Vê se já tem a resposta. Se tiver, pula e passa para a próxima
    if "resposta_obtida" not in perg_resp.keys():
      resposta_obtida, historico_mensagens = chat_completion_react(pergunta, max_mensagens=10, sleep=20, debug=True)
      perg_resp_esperada_obtida.append({
        "pergunta": pergunta,
        "resposta_esperada": resposta_esperada,
        "resposta_obtida": resposta_obtida,
        "historico_mensagens": historico_mensagens
      })
    salvar_perg_resp_esperada_obtida(perg_resp_esperada_obtida)
else:
  perg_resp_esperada_obtida = recuperar_perg_resp_esperada_obtida()

******************* 0/50 *******************
Pergunta: What is Zeus know for in Greek mythology?
Ação: Search. Parâmetros: Zeus Greek mythology
Ação: Search. Parâmetros: Zeus Greek mythology king of gods
Ação: Finish. Parâmetros: king of the gods of Mount Olympus and the sky and thunder god
******************* 1/50 *******************
Pergunta: How long had the First World War been over when Messe was named aide-de-camp?
Ação: Search. Parâmetros: Messe aide-de-camp
Ação: Search. Parâmetros: Messe First World War
Ação: Search. Parâmetros: Messe
Ação: Search. Parâmetros: Messe aide-de-camp World War I
Ação: Search. Parâmetros: Messe aide-de-camp First World War general
Ação: Search. Parâmetros: Messe aide-de-camp World War I general officer
Ação: Search. Parâmetros: Messe aide-de-camp World War I general officer Italian
Ação: Finish. Parâmetros: After World War I had ended
******************* 2/50 *******************
Pergunta: How old was Messe when the First World War started?
Ação: Sea

In [174]:
import time
import json


for i, perg_resp in enumerate(perg_resp_esperada_obtida):
  print(f'******************* {i}/{len(perg_resp_esperada_obtida)} *******************')
  pergunta = perg_resp['pergunta']
  resposta_esperada = perg_resp['resposta_esperada']
  resposta_obtida = perg_resp['resposta_obtida']

  print(pergunta)
  print(resposta_esperada)
  print(resposta_obtida)

******************* 0/50 *******************
What is Zeus know for in Greek mythology?
sky and thunder god
king of the gods of Mount Olympus and the sky and thunder god
******************* 1/50 *******************
How long had the First World War been over when Messe was named aide-de-camp?
5 years
After World War I had ended
******************* 2/50 *******************
How old was Messe when the First World War started?
30 years
I need more information about Max Müller's birthdate to answer the question
******************* 3/50 *******************
How long had Angela Scoular been acting professionally when she appeared in the movie "On Her Majesty's Secret Service"?
2 years
around 7-9 years
******************* 4/50 *******************
What is the capacity of the stadium where Brunt returned to action after a torn ACL?
26,688
MCG
******************* 5/50 *******************
Which stadium where Brunt played can hold more people, The Hawthorns or White Hart Lane?
White Hart Lane
The Hawt

# Cálculo do F1

In [165]:
# Funções normize_answer e token_f1_score foram retiradas do Visconde (arquivo qasper_evaluator.py)
import re
import string

def normalize_answer(s):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  Lower text and remove punctuation, articles and extra whitespace.
  """

  def remove_articles(text):
      return re.sub(r"\b(a|an|the)\b", " ", text)

  def white_space_fix(text):
      return " ".join(text.split())

  def remove_punc(text):
      exclude = set(string.punctuation)
      return "".join(ch for ch in text if ch not in exclude)

  def lower(text):
      return text.lower()

  return white_space_fix(remove_articles(remove_punc(lower(s))))

def token_f1_score(prediction, ground_truth):
  """
  Taken from the official evaluation script for v1.1 of the SQuAD dataset.
  """
  prediction_tokens = normalize_answer(prediction).split()
  ground_truth_tokens = normalize_answer(ground_truth).split()
  common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
  num_same = sum(common.values())
  if num_same == 0:
      return 0
  precision = 1.0 * num_same / len(prediction_tokens)
  recall = 1.0 * num_same / len(ground_truth_tokens)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

In [168]:
import pandas as pd

col_queries = []
col_resp_esperadas = []
col_historico_mensagens = []
col_respostas_llm = []
col_f1_score = []
col_em_score = []

for perg_respostas in perg_resp_esperada_obtida:
  query = perg_respostas['pergunta']
  resposta_esperada = normalize_answer(perg_respostas['resposta_esperada'])
  resposta_llm = normalize_answer(perg_respostas['resposta_obtida'])
  historico_mensagens = perg_respostas['historico_mensagens']

  f1_score = token_f1_score(resposta_llm, resposta_esperada)
  em_score = 1 if resposta_llm == resposta_esperada else 0

  col_queries.append(query)
  col_resp_esperadas.append(resposta_esperada)
  col_respostas_llm.append(resposta_llm)
  col_historico_mensagens.append(historico_mensagens)
  col_f1_score.append(f1_score)
  col_em_score.append(em_score)

df = pd.DataFrame({'Query': col_queries, 'Resp_esperada': col_resp_esperadas, 'Resposta_LLM': col_respostas_llm, 'F1_score': col_f1_score, 'EM_score': col_em_score, 'Historico_msg': col_historico_mensagens})
df

,Query,Resp_esperada,Resposta_LLM,F1_score,EM_score,Historico_msg
0,What is Zeus know for in Greek mythology?,sky and thunder god,king of gods of mount olympus and sky and thun...,0.533333,0,"[{'role': 'system', 'content': 'You are a help..."
1,How long had the First World War been over whe...,5 years,after world war i had ended,0.000000,0,"[{'role': 'system', 'content': 'You are a help..."
2,How old was Messe when the First World War sta...,30 years,i need more information about max müllers birt...,0.000000,0,"[{'role': 'system', 'content': 'You are a help..."
3,How long had Angela Scoular been acting profes...,2 years,around 79 years,0.400000,0,"[{'role': 'system', 'content': 'You are a help..."
4,What is the capacity of the stadium where Brun...,26688,mcg,0.000000,0,"[{'role': 'system', 'content': 'You are a help..."
5,Which stadium where Brunt played can hold more...,white hart lane,hawthorns with capacity of 26688 has smaller c...,0.272727,0,"[{'role': 'system', 'content': 'You are a help..."
6,In which country was Wilhelm Müller born?,germany,germany,1.000000,1,"[{'role': 'system', 'content': 'You are a help..."
7,Which battle Wilhelm Müller fought in while in...,kulm,john george ii prince of anhaltdessau or dietr...,0.000000,0,"[{'role': 'system', 'content': 'You are a help..."
8,How much time had passed between Wilhelm's ret...,9 years,exact time is not specified but wilhelm müller...,0.071429,0,"[{'role': 'system', 'content': 'You are a help..."
9,Which of Wilhelm's direct male descendants liv...,friedrich max müller,wilhelm max müller,0.666667,0,"[{'role': 'system', 'content': 'You are a help..."


Exibe o F1 e o EM (exact match) médios:

In [169]:
print(f'Total de exemplos: {len(df)}')
print(f"F1 score. Média: {df['F1_score'].mean():.2f}. Std: {df['F1_score'].std():.2f}")
print(f"EM score. Média: {df['EM_score'].mean():.2f}. Std: {df['EM_score'].std():.2f}")

Total de exemplos: 50
F1 score. Média: 0.35. Std: 0.41
EM score. Média: 0.24. Std: 0.43


Fazendo a inspeção manual das respostas, 21/50 estão corretas (0.42).